#Set-up dependencies and API key

In [5]:
%pip install -qU python-dotenv
%pip install -qU langchain
%pip install -qU langchain_openai
%pip install -qU langchain_community
%pip install -qU langchain-huggingface
%pip install -qU langchain-google-genai
%pip install -qU streamlit
%pip install -qU faiss-cpu
%pip install -qU sentence-transformers
%pip install -qU pypdf
%pip install -qU docx2txt


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
#Google API Key
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now you can access the API key like this
google_api_key = os.getenv("GOOGLE_API_KEY")

# Chatbot RAG

## Xây dựng Vector database

In [ ]:
#Load data from folder
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
file_path = "./docs/PBL6.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

In [8]:
print(docs[0].page_content[:200])

ĐẠI HỌC ĐÀ NẴNG 
ĐẠI HỌC BÁCH KHOA 
KHOA CÔNG NGHỆ THÔNG TIN 
 
 
 
 
 
PBL6: DỰ ÁN CHUYÊN NGÀNH 
CÔNG NGHỆ PHẦN MỀM 
Đề tài: HỆ THỐNG NGHE NHẠC TRỰC TUYỂN 
MELODIES 
GIẢNG VIÊN HƯỚNG DẪN: ThS. Mai Vă


In [9]:
from langchain_text_splitters.character import  RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=400,
      chunk_overlap=200
)
doc_chunks = text_splitter.split_documents(docs)
#Embedding các chunks này và đưa vào vector store
embeddings = HuggingFaceEmbeddings()
vectordb = FAISS.from_documents(doc_chunks, embeddings)

c:\Users\fredd\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Lưu vector store
vectordb.save_local("index")

## Xây dựng chain

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
retriever = vectordb.as_retriever(
        search_kwargs={
                  "k": 2, # sl kq truy vấn trả về
    }
    )
memory = ConversationBufferMemory(
        llm=llm,
        output_key="answer",
        memory_key="chat_history",
        return_messages=True
    )
chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        chain_type="map_reduce",
        memory=memory,
        verbose=True
    )

C:\Users\fredd\AppData\Local\Temp\ipykernel_7992\1189939838.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [1]:
memory.clear()
while True:
    user_input = input("user: ")
    if user_input.lower() == "exit":
        break

    response = chain({"question": user_input, "chat_history": memory.chat_memory.messages})
    assistant_response = response["answer"]

    print("assistant: ", assistant_response)

NameError: name 'memory' is not defined

In [ ]:
memory.chat_memory.messages

[HumanMessage(content=' Nội dung của khóa học lập trình python cơ bản ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Khóa học dạy học viên sử dụng cấu trúc trong Python để xây dựng ứng dụng đơn giản.  Nó cũng rèn luyện kỹ năng lập trình, tư duy logic, và xây dựng nền tảng cơ bản trong Python để học về lập trình web, game, desktop, machine learning, và data science.  Tuy nhiên, nội dung khóa học không được mô tả đầy đủ.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Mục tiêu của nó là gì ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Mục tiêu của khóa học là giúp học viên hiểu lập trình Python để ứng dụng trong công việc, học tập và cuộc sống;  sử dụng các cấu trúc Python để xây dựng ứng dụng; rèn luyện kỹ năng lập trình và tư duy logic; và xây dựng nền tảng vững chắc trong Python để học các kiến thức lập trình Web, Game, Desktop, Machine Learning, Data Science.\n', additional_kwargs={}, response_metadata={})]